In [ ]:
# Sentiment Analyzer using LSTM 
# and IMDB movie reviews
# to use this dataset please upload the dataset to your google drive
#References:
#https://keras.io/api/layers/activations/
#https://keras.io/api/
#https://keras.io/api/models/sequential/

from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm
!pip install pyswarms

     |████████████████████████████████| 2.1 MB 5.1 MB/s 
     |████████████████████████████████| 1.1 MB 42.6 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 653.3/653.3 KB 39.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 KB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 KB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.1/457.1 KB 31.6 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.2.0
    Uninstalling typing_extensions-4.2.0:
      Successfully uninstalled typing_extensions-4.2.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempt

In [ ]:
import os
import random
import pandas as pd
import numpy as np
from collections import Counter
import re
import nltk
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.model_selection import train_test_split

# Import the dataset
#helper functions
def readFile(file_path):
  with open(file_path, 'r') as file:
    return file.read()

def create_set(path, sentiment):
  os.chdir(path)
  entries = []
  labels = []
  for file in os.listdir():
    if file.endswith(".txt"):
      file_path = f"{path}/{file}"
      #annotate the data
      if sentiment == 0:
        labels.append(0)
      else:
        labels.append(1)
      entries.append(readFile(file_path))
  
  return entries, labels

#get the paths for the training set first
# path_to_train_pos = '/content/drive/MyDrive/Datasets/train/pos/half_pos'
# path_to_train_neg = '/content/drive/MyDrive/Datasets/train/neg/half_neg'

# Jakob's path:
path_to_train_pos = '/content/drive/MyDrive/College_Docs/Natural_Language_Processing/Datasets/train/pos/half_pos'
path_to_train_neg = '/content/drive/MyDrive/College_Docs/Natural_Language_Processing/Datasets/train/neg/half_neg'


train_pos_text, tpos_labels = create_set(path_to_train_pos, 0)
train_neg_text, tneg_labels = create_set(path_to_train_neg, 1)

train_pos = list(zip(train_pos_text, tpos_labels))
train_neg = list(zip(train_neg_text, tneg_labels))

#get equal amounts of both pos and neg samples
#sample numbers have to match half of what you want the total to be
train_pos = train_pos[:3000]
train_neg = train_neg[:3000]

train_list = train_pos + train_neg

random.shuffle(train_list)

train_df = pd.DataFrame(train_list, columns=['text', 'label'])

#then the test set

train_df, test_df = train_test_split(train_df, test_size= 0.33, shuffle=True)


In [ ]:
print(test_df)

                                                   text  label
610   This agonizing comedy-drama got surprisingly s...      1
4270  I question anyone saying they don't care for t...      0
3707  When I rented this movie to watch it, I knew t...      0
2534  This is undoubtedly one of the funniest movies...      0
1420  What I expected: A rather lame overly-stereoty...      0
...                                                 ...    ...
1697  Curl up with this one on a dark and stormy nig...      0
3888  A very slick modern (keeping it sensually hip)...      1
4977  Greetings again from the darkness. 18 director...      0
478   SOLDIER is not as bad as many have made it out...      0
377   The ENTIRE MOVIE is flashbacks from the first ...      1

[1980 rows x 2 columns]


In [ ]:
import gensim
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_words = list(stopwords.words('english'))
def clean_text(text):
    # Remove new line characters
    text = re.sub('\s+', ' ', text)
    # Remove distracting single quotes
    text = re.sub("\'", "", text)

    text = re.sub(r"\d", '', text)

    text = re.sub(r"[^\w\s]", '', text)
    return text

# Remove repeating words
def remove_repeats(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

# Detokenize the text to use word embeddings
def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

count = 0
temp = []
#Splitting pd.Series to list
data_to_list = train_df['text'].values.tolist()
for word in data_to_list:
  if not word.lower() in stop_words:
    temp.append(clean_text(word).lower())
data_words = list(remove_repeats(temp))

x_train = []
for word in data_words:
    x_train.append(detokenize(word))

temp = []
#Splitting pd.Series to list
data_to_list = test_df['text'].values.tolist()
for word in data_to_list:
  if not word.lower() in stop_words:
    temp.append(clean_text(word).lower())
data_words = list(remove_repeats(temp))


x_test = []
for word in data_words:
    x_test.append(detokenize(word))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
len(x_train[1])

731

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers

max_words = 1000
max_len = 500

tokenizer_train = Tokenizer(num_words=max_words)
tokenizer_train.fit_on_texts(x_train)
sequences_train = tokenizer_train.texts_to_sequences(x_train)
reviews_train = pad_sequences(sequences_train, maxlen=max_len)


tokenizer_test = Tokenizer(num_words=max_words)
tokenizer_test.fit_on_texts(x_test)
sequences_test = tokenizer_test.texts_to_sequences(x_test)
reviews_test = pad_sequences(sequences_test,maxlen=max_len)


In [ ]:
import tensorflow as tf

tr_labels = np.array(train_df['label'].values)
y_train = tf.keras.utils.to_categorical(tr_labels, 2, dtype="float32")
print(type(y_train))

ts_labels = np.array(test_df['label'].values)
y_test = tf.keras.utils.to_categorical(ts_labels, 2, dtype="float32")
print(type(ts_labels))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [ ]:
import keras
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.layers import SpatialDropout1D



lstm = Sequential()
lstm.add(layers.Embedding(max_words+1, 64))
lstm.add(SpatialDropout1D(0.4))
lstm.add(layers.LSTM(64,dropout=0.3, recurrent_dropout =0.3))
lstm.add(layers.Dense(2,activation='sigmoid'))

optimizer= tf.keras.optimizers.SGD(learning_rate=0.001,)

lstm.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=['accuracy'])

checkpoint1 = ModelCheckpoint("best_model1.hdf5", monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', save_freq=15,save_weights_only=False)

lstm.summary()
lstm.count_params()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 64)          64064     
                                                                 
 spatial_dropout1d (SpatialD  (None, None, 64)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 2)                 130       
                                                                 
Total params: 97,218
Trainable params: 97,218
Non-trainable params: 0
_________________________________________________________________


97218

In [ ]:
print(reviews_train.shape)
print(reviews_test.shape)
print(y_train.shape)
print(y_test.shape)
history_1 = lstm.fit(reviews_train, y_train, validation_data=(reviews_test, y_test),epochs=5, callbacks=[checkpoint1], batch_size=60)

(4020, 500)
(1980, 500)
(4020, 2)
(1980, 2)
Epoch 1/5
67/67 [==============================] - 106s 2s/step - loss: 0.6931 - accuracy: 0.5005 - val_loss: 0.6930 - val_accuracy: 0.5045
Epoch 2/5
67/67 [==============================] - 90s 1s/step - loss: 0.6932 - accuracy: 0.4948 - val_loss: 0.6930 - val_accuracy: 0.5167
Epoch 3/5
67/67 [==============================] - 89s 1s/step - loss: 0.6931 - accuracy: 0.5067 - val_loss: 0.6930 - val_accuracy: 0.5121
Epoch 4/5
67/67 [==============================] - 89s 1s/step - loss: 0.6930 - accuracy: 0.5109 - val_loss: 0.6930 - val_accuracy: 0.5146
Epoch 5/5
67/67 [==============================] - 90s 1s/step - loss: 0.6931 - accuracy: 0.5030 - val_loss: 0.6930 - val_accuracy: 0.5192


In [ ]:
#testing the test set
score, acc = lstm.evaluate(reviews_test, y_test, verbose=2, batch_size=120)
print("score: %.2f" % (1-score))
print("Accuracy: %.2f" % (acc))

17/17 - 4s - loss: 0.6930 - accuracy: 0.5192 - 4s/epoch - 226ms/step
score: 0.31
Accuracy: 0.52


# Particle Swarm Optimization

In [ ]:
import pyswarms as ps

# Extract the shape of the model for reassignment to be
# compatible with PSO.
def get_model_struct(lstm):
  weights_list = lstm.get_weights()
  structure = []

  # Get the shape of all the model's weights
  for weight in weights_list:
    structure.append(weight.shape)
  return structure

# Reset model structure for PSO compatibility.
def set_PSO_struct(weights_list, structure):
  new_weights = []
  i = 0
  
  # For each layer 
  for struct in structure:

    # Have a set of particles for the number of parameters 
    if(len(struct)>1):
      num_nodes = np.prod(struct) + i
    else:
      num_nodes = struct[0] + i
    
    restructured_weights = np.array(weights_list[i:num_nodes]).reshape(struct)
    new_weights.append(restructured_weights)
    i = num_nodes

  return new_weights

In [ ]:
def PSO_evaluation(weights, struct, lstm, reviews_train=reviews_train, y_train=y_train):
  results = []
  for weight in weights:
    lstm.set_weights(set_PSO_struct(weight, struct))
    score = lstm.evaluate(reviews_train, y_train)
    results.append(1-score[1])
  return results

In [ ]:
# Extract the model's structure
structure = get_model_struct(lstm)

# Define the upper and lower boundaries for PSO parameters
# since the particles are weights and weights are between
# -1 and 1.
up_bound = np.ones(lstm.count_params())
low_bound = -1.0 * np.ones(lstm.count_params())
boundaries = (low_bound, up_bound)

# Hyperparameters that need tuning
hyperparameters = {'c1': 1000, 'c2': 800, 'w': 500}

# Build the Global optimizer
PSO = ps.global_best.GlobalBestPSO(n_particles=20, dimensions=lstm.count_params(), options=hyperparameters, bounds=boundaries)

# Model Implementation

In [ ]:
# 
cost, pos = PSO.optimize(PSO_evaluation, 5, reviews_train=reviews_train, y_train=y_train, struct=structure, lstm=lstm)
lstm.set_weights(set_PSO_struct(pos, structure))
score = lstm.evaluate(reviews_test, y_test)

print(score[0])
print(score[1])
# history = lstm.fit(reviews_train, y_train, batch_size=50, epochs=5, validation_data=(reviews_test, y_test),callbacks=[checkpoint1])

2022-04-29 00:28:27,847 - pyswarms.single.global_best - INFO - Optimize for 5 iters with {'c1': 1000, 'c2': 800, 'w': 500}
pyswarms.single.global_best:   0%|          |0/5

 98/126 [======================>.......] - ETA: 5s - loss: 1.1908 - accuracy: 0.4990

pyswarms.single.global_best:   0%|          |0/5


KeyboardInterrupt: ignored

In [ ]:
lstm.evaluate(reviews_test, y_test)